In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20000)
pd.options.mode.chained_assignment = None #Ignore Error: SettingWithCopyWarning
import math
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import os
import keras
import tensorflow as tf
from keras.layers import Input, Flatten, Dense, Lambda, Reshape, Dropout
from keras.layers import BatchNormalization
from keras.models import Model
from keras.losses import binary_crossentropy
from keras.metrics import mse, accuracy
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K

keras.backend.clear_session()

# Load the data

In [8]:
#df = pd.read_csv('../data/preprocessed/cleaned_data_v7.csv', delimiter=",")
df = pd.read_csv('../data/preprocessed/cleaned_data_agg_v1.csv', delimiter=",")

key_list = df.keys().tolist()

for x in range(len(key_list)):
    if not df.columns[x] == "datum":
        if not df.columns[x] == "container_id":
            df = df.astype({key_list[x]: "float32"})

# Delete unwanted data; cast datetime

In [9]:
df = df.drop(columns=["height_in_cm"])

# set date as index ancd convert to float
df['datum'] = pd.to_datetime(df['datum'])
df = df.set_index(df['datum'])
df = df.sort_index()
df['datum_float'] = df['datum'].values.astype(float)

# Split into train/test

In [10]:
# create train test partition
training_set = df['2020-05-09 00:00:00':'2021-01-29 23:59:59']
test_set  = df['2021-01-30 00:00:00':]
print('Train Dataset:',training_set.shape)
print('Test Dataset:',test_set.shape)

Train Dataset: (19142, 25)
Test Dataset: (7539, 25)


<ipython-input-10-4a93c0fbbe20>:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  training_set = df['2020-05-09 00:00:00':'2021-01-29 23:59:59']
<ipython-input-10-4a93c0fbbe20>:3: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  test_set  = df['2021-01-30 00:00:00':]


In [11]:
training_set

,container_id,temperatur_in_grad,aussentemperatur,niederschlag_in_mm,ist_industriegebiet,ist_wohngebiet,ist_feiertag,datum,sekunde,minute,stunde,wochentag,kalenderwoche,monat,jahr,farbe_brown,farbe_green,farbe_white,unix_time,time_delta,hight_delta,ist_ferien,laengengrad,breitengrad,datum_float
datum,,,,,,,,,,,,,,,,,,,,,,,,,
2020-05-09 19:40:46.486563,70.0,17.500000,18.950001,0.0,1.0,0.0,0.0,2020-05-09 19:40:46.486563,46.0,40.0,19.0,5.0,19.0,5.0,2020.0,0.0,1.0,0.0,1.589050e+12,50399600.0,-22.0,0.0,50.119270,8.770680,1.589053e+18
2020-05-09 21:22:06.657404,57.0,20.904762,8.366667,0.6,1.0,0.0,0.0,2020-05-09 21:22:06.657404,6.0,22.0,21.0,5.0,19.0,5.0,2020.0,0.0,1.0,0.0,1.589050e+12,3599860.0,-180.0,0.0,50.123043,8.765538,1.589059e+18
2020-05-09 22:42:23.140342,25.0,18.181818,23.118181,0.0,0.0,1.0,0.0,2020-05-09 22:42:23.140342,23.0,42.0,22.0,5.0,19.0,5.0,2020.0,0.0,0.0,1.0,1.589060e+12,7199647.0,-24.0,0.0,50.130257,8.709095,1.589064e+18
2020-05-09 22:47:38.461644,71.0,14.800000,4.095000,0.6,0.0,1.0,0.0,2020-05-09 22:47:38.461644,38.0,47.0,22.0,5.0,19.0,5.0,2020.0,1.0,0.0,0.0,1.589060e+12,3599811.0,-106.0,0.0,50.125187,8.710195,1.589064e+18
2020-05-09 22:49:15.517636,6.0,14.523809,1.509524,0.6,0.0,1.0,0.0,2020-05-09 22:49:15.517636,15.0,49.0,22.0,5.0,19.0,5.0,2020.0,1.0,0.0,0.0,1.589060e+12,3599771.0,-2.0,0.0,50.134075,8.707200,1.589065e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-29 23:47:57.755353,37.0,7.842105,9.321053,5.5,0.0,1.0,0.0,2021-01-29 23:47:57.755353,57.0,47.0,23.0,4.0,4.0,1.0,2021.0,1.0,0.0,0.0,1.611960e+12,3599852.0,-18.0,0.0,50.129505,8.706566,1.611964e+18
2021-01-29 23:53:48.114059,34.0,11.608696,11.126087,9.2,0.0,1.0,0.0,2021-01-29 23:53:48.114059,48.0,53.0,23.0,4.0,4.0,1.0,2021.0,0.0,1.0,0.0,1.611960e+12,3599919.0,-20.0,0.0,50.128185,8.704522,1.611964e+18
2021-01-29 23:55:40.509137,55.0,14.055555,5.477778,8.3,0.0,1.0,0.0,2021-01-29 23:55:40.509137,40.0,55.0,23.0,4.0,4.0,1.0,2021.0,0.0,0.0,1.0,1.611960e+12,3599861.0,-46.0,0.0,50.133209,8.697432,1.611965e+18


# Prepare the training data

In [12]:
#https://towardsdatascience.com/lstm-time-series-forecasting-predicting-stock-prices-using-an-lstm-model-6223e9644a2f
#https://analyticsindiamag.com/hands-on-guide-to-lstm-recurrent-neural-network-for-stock-market-prediction/
# Feature Scaling

# ****************** START TUNING *************
timelag = 3 #10
epochs_number = 1000 #10
batch_size_number = 32 #32
container_number = 1

# ****************** END TUNING *************

training_set = training_set[training_set['container_id']==container_number]
train_set = training_set.iloc[:, 20:21].values

sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(train_set)
X_train = []
y_train = []
for i in range(timelag, len(training_set)-1):
    X_train.append(training_set_scaled[i-timelag:i, 0])
    y_train.append(training_set_scaled[i, 0]) 
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [18]:
y_train

array([1.        , 1.        , 0.9830508 , 0.9830508 , 0.9830508 ,
       1.        , 1.        , 0.9830508 , 1.        , 0.9830508 ,
       0.7966102 , 0.86440676, 0.7118644 , 0.7118644 , 0.91525424,
       0.9322034 , 0.6949153 , 0.89830506, 0.88135594, 0.9322034 ,
       0.6779661 , 0.8135593 , 0.86440676, 0.779661  , 0.91525424,
       0.88135594, 0.62711865, 0.7627119 , 0.779661  , 0.88135594,
       0.7966102 , 0.7627119 , 0.84745765, 0.7966102 , 0.779661  ,
       0.86440676, 0.7966102 , 0.89830506, 0.7966102 , 0.9322034 ,
       0.84745765, 0.9322034 , 0.88135594, 0.7966102 , 0.9830508 ,
       0.9661017 , 1.        , 0.9322034 , 0.89830506, 0.8135593 ,
       0.8305085 , 0.84745765, 0.779661  , 0.86440676, 0.779661  ,
       0.89830506, 0.91525424, 0.91525424, 0.8305085 , 0.779661  ,
       0.5762712 , 0.6949153 , 0.89830506, 0.9491525 , 0.8135593 ,
       0.84745765, 0.89830506, 0.86440676, 0.7118644 , 0.84745765,
       0.8305085 , 0.8305085 , 0.89830506, 0.84745765, 0.83050

# Configure the model

In [7]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

#first callback method
#reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, verbose=1,
#patience=25, min_lr=0.0000001, min_delta=0.0001)
#default value lr = 0.001
#callbacks = [reduce_lr]

# Second callback mehod
#es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Train the model

In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
history = regressor.fit(X_train, y_train, epochs = epochs_number, batch_size = batch_size_number)
#, callbacks = callbacks
# callbacks=[es_callback]
plt.figure(figsize=(20,10))
plt.plot(history.history['loss'], label='train')
plt.legend()
plt.show()

Epoch 1/1000
173/173 [==============================] - 51s 20ms/step - loss: 0.3129
Epoch 2/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0287
Epoch 3/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0269
Epoch 4/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0256
Epoch 5/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0307
Epoch 6/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0288: 0
Epoch 7/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0228
Epoch 8/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0274: 0s - los
Epoch 9/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0266
Epoch 10/1000
173/173 [==============================] - 3s 19ms/step - loss: 0.0256
Epoch 11/1000
173/173 [==============================] - 3s 19ms/step - loss: 0.0288: 0s - lo
Epoch 12/1000
173/173 [============================

173/173 [==============================] - 4s 21ms/step - loss: 0.0230: 2s
Epoch 189/1000
173/173 [==============================] - 4s 21ms/step - loss: 0.0232: 1s - loss: 0.024 - ET
Epoch 190/1000
173/173 [==============================] - 4s 22ms/step - loss: 0.0223
Epoch 191/1000
173/173 [==============================] - 4s 23ms/step - loss: 0.0227
Epoch 192/1000
173/173 [==============================] - 4s 21ms/step - loss: 0.0209
Epoch 193/1000
173/173 [==============================] - 4s 21ms/step - loss: 0.0245
Epoch 194/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0220
Epoch 195/1000
173/173 [==============================] - 3s 19ms/step - loss: 0.0210
Epoch 196/1000
173/173 [==============================] - 3s 19ms/step - loss: 0.0247
Epoch 197/1000
173/173 [==============================] - 3s 19ms/step - loss: 0.0219
Epoch 198/1000
173/173 [==============================] - 3s 19ms/step - loss: 0.0210
Epoch 199/1000
173/173 [==================

173/173 [==============================] - 4s 23ms/step - loss: 0.0220: 0s
Epoch 374/1000
173/173 [==============================] - 4s 23ms/step - loss: 0.0210
Epoch 375/1000
173/173 [==============================] - 4s 21ms/step - loss: 0.0242
Epoch 376/1000
173/173 [==============================] - 4s 22ms/step - loss: 0.0212
Epoch 377/1000
173/173 [==============================] - 4s 20ms/step - loss: 0.0220
Epoch 378/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0216
Epoch 379/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0228
Epoch 380/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0213
Epoch 381/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0204
Epoch 382/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0211
Epoch 383/1000
173/173 [==============================] - 3s 20ms/step - loss: 0.0215
Epoch 384/1000
173/173 [==============================] - 3s 20ms

# Prepare the test data

In [ ]:
#test_set_sub = test_set
test_set_sub = test_set[test_set['container_id']==container_number]
#training_set_sub = training_set
training_set_sub = training_set[training_set['container_id']==container_number]

real_height = test_set_sub.iloc[:, 20:21].values
dataset_total = pd.concat((training_set_sub['hight_delta'], test_set_sub['hight_delta']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(test_set_sub) - timelag:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(timelag, len(test_set_sub)+timelag):
    X_test.append(inputs[i-timelag:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Predict

In [ ]:
predicted_height = regressor.predict(X_test)
predicted_height = sc.inverse_transform(predicted_height)

# Plot the prediction

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(real_height[100:300], color = 'green', label = 'Real Height')
plt.plot(predicted_height[100:300], color = 'red', label = 'Predicted Height')
plt.title('Height Prediction with Timelag '+ str(timelag) + ' for container number ' +str(container_number))
plt.xlabel('Time')
plt.ylabel('Height Delta')
plt.legend()
plt.show()

# Export Model

In [ ]:
regressor.save('../data/modeling/dev/test_model')

# TEST FOR A SINGLE PREDICTION

In [ ]:
reconstructed_model = keras.models.load_model("../data/modeling/dev/test_model")

In [ ]:
test_set_sub = test_set[test_set['container_id']==1].head(1)
training_set_sub = training_set[training_set['container_id']==1]

real_height = test_set_sub.iloc[:, 20:21].values
dataset_total = pd.concat((training_set_sub['hight_delta'], test_set_sub['hight_delta']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(test_set_sub) - timelag:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(timelag, len(test_set_sub)+timelag):
    X_test.append(inputs[i-timelag:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

predicted_height = reconstructed_model.predict(X_test)
predicted_height = sc.inverse_transform(predicted_height)

print("Pred ="+str(predicted_height)+" and real height = "+str(real_height))

In [ ]:
test_set_sub